## Stability of ODE methods for the heat equation

In [1]:
import mfem.ser as mfem
import numpy as np
import time
from glvis import glvis

In [2]:
order = 1
mesh = mfem.Mesh(20, 20, "TRIANGLE")
dim = mesh.Dimension()

fec = mfem.H1_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)

print("Number of unknowns: " + str(fes.GetVSize()))

def s1d(k, x):
    return np.sin(2*k*np.pi*x)
def u1d(k, x):
    skx = s1d(k, x)
    if skx == 0.0:
        return 0.0
    else:
        return np.exp(-1.0 / (skx**2)) * np.sign(skx)
def w1d(n, x):
    val = 0.0
    for j in range(0, n):
        val += u1d(3**j, x)
    return val

class InitialCondition(mfem.PyCoefficient):
    def EvalValue(self, xvec):
        val = 1.0
        for x in xvec:
            val *= w1d(1, x)
        return val

ic_coeff = InitialCondition()

u = mfem.GridFunction(fes)
u.ProjectCoefficient(ic_coeff)

Number of unknowns: 441


In [3]:
glvis((mesh, u))

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [4]:
one = mfem.ConstantCoefficient(1.0)

m = mfem.BilinearForm(fes)
m.AddDomainIntegrator(mfem.MassIntegrator())
k = mfem.BilinearForm(fes)
k.AddDomainIntegrator(mfem.DiffusionIntegrator())
b = mfem.LinearForm(fes)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(one))

m.Assemble()
m.Finalize()
k.Assemble()
k.Finalize()
b.Assemble()

ess_dofs = mfem.intArray()
ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
fes.GetEssentialTrueDofs(ess_bdr, ess_dofs)

M = mfem.OperatorPtr()
K = mfem.OperatorPtr()
m.FormSystemMatrix(ess_dofs, M)
k.FormSystemMatrix(ess_dofs, K)


### Solve and Plot the Solution with GLVis

In [5]:
u.ProjectCoefficient(ic_coeff)

g = glvis((mesh, u))
g.render()

class FE_Evolution(mfem.PyTimeDependentOperator):
    def __init__(self, M, K, b):
        mfem.PyTimeDependentOperator.__init__(self, M.Size())
        self.K = K
        self.M = M
        self.b = b
        self.z = mfem.Vector(M.Size())
        self.zp = np.zeros(M.Size())
        self.M_prec = mfem.DSmoother()
        self.M_solver = mfem.CGSolver()
        self.M_solver.SetPreconditioner(self.M_prec)
        self.M_solver.SetOperator(M)
        self.M_solver.iterative_mode = False
        self.M_solver.SetRelTol(1e-9)
        self.M_solver.SetMaxIter(1000)
        self.M_solver.SetPrintLevel(0)

        self.A_prec = mfem.GSSmoother()
        self.A_solver = mfem.CGSolver()
        self.A_solver.SetPreconditioner(self.A_prec)
        self.A_solver.iterative_mode = False
        self.A_solver.SetRelTol(1e-9)
        self.A_solver.SetMaxIter(1000)
        self.A_solver.SetPrintLevel(0)

    def Mult(self, x, y):
        self.K.Mult(x, self.z)
        self.z *= -1.0
        self.z += b
        self.M_solver.Mult(self.z, y)

    def ImplicitSolve(self, gamma, u, k):
        B = mfem.Add(1.0, self.M, gamma, self.K)
        self.A_solver.SetOperator(B)
        self.K.Mult(u, self.z)
        self.z *= -1.0
        self.z += b
        self.A_solver.Mult(self.z, k)


heat_eq = FE_Evolution(mfem.OperatorHandle2SparseMatrix(M), mfem.OperatorHandle2SparseMatrix(K), b)

# ode_solver = mfem.ForwardEulerSolver()
ode_solver = mfem.BackwardEulerSolver()
# ode_solver = mfem.TrapezoidalRuleSolver() # Crank-Nicolson
ode_solver.Init(heat_eq)

t = 0.0
ti = 0
dt = 1e-3
t_final = 1e-1
vis_steps = 5
end = False

time.sleep(1)
while not end:
    if t > t_final - dt/2: break
    t, dt = ode_solver.Step(u, t, dt);
    ti = ti + 1
    if u.Normlinf() > 1e4:
        print("Unstable!")
        end = True
    if ti % vis_steps == 0 or end:
        g.update((mesh, u))
        time.sleep(0.05)
        print(f"time step: {ti}, time: {t:.2e}, {u.CheckFinite()}", end="\r")

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…